### Подключаем библиотеки

### Задаём необходимые функции

In [1]:
# Подключаем библиотеки
import cv2
import matplotlib.pyplot as plt

from google.colab import drive
import tqdm
import numpy as np
import os

from sklearn.svm import SVC
import pickle
from joblib import dump, load

from skimage import feature
from skimage import exposure
from skimage import feature

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
drive.mount('/content/drive')
WORKING_DIRECTORY = '/content/drive/My Drive/nti-logo-recognition/'
os.chdir(WORKING_DIRECTORY)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Извлечение HOG из картинки
# Библиотеки для функции HOG


# Параметры HOG
_orientations = 15 # количество направлений градиентов
_pixels_per_cell = (16, 16) # количество пикселей на клетку
_cells_per_block = (4, 4) # Количество клеток на блок
_transform_sqrt = True # Уборка шумов с помощью корня
_block_norm = "L1" # Метод нормализации блока
_visualize = True # Визуализация
settings = [_orientations, _pixels_per_cell, _cells_per_block, _transform_sqrt, _block_norm, _visualize]

# Параметры обработки входных изображений
IMG_HEIGHT = 200 # Высота
IMG_WIDTH = 200 # Ширина

# Функция извлечение карты HOG и её изображения
def GetHOG (img, settings): # Входное изображеине
    HOG, img_HOG = feature.hog(img, 
                               orientations = settings[0], 
                               pixels_per_cell = settings[1], 
                               cells_per_block = settings[2], 
                               transform_sqrt = settings[3], 
                               block_norm = settings[4],
                               visualize = settings[5])
    return (HOG, img_HOG)

In [ ]:
# Функция конвертации цвета
def convert(frame, src_model = "rgb", dest_model = "hls"):
    
    if src_model == "rgb" and dest_model == "hsv": 
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    elif src_model == "rgb" and dest_model == "hls":
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HLS)
    elif src_model == "rgb" and dest_model == "yuv":
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2YUV)
    elif src_model == "rgb" and dest_model == "ycrcb":
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2YCR_CB)
    elif src_model == "hsv" and dest_model == "rgb":
      frame = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)
    elif src_model == "hls" and dest_model == "rgb":
      frame = cv2.cvtColor(frame, cv2.COLOR_HLS2RGB)
    elif src_model == "yuv" and dest_model == "yuv":
      frame = cv2.cvtColor(frame, cv2.COLOR_YUV2RGB)
    elif src_model == "ycrcb" and dest_model == "ycrcb":
      frame = cv2.cvtColor(frame, cv2.COLOR_YCR_CB2RGB)
    elif (src_model == "rgb" and dest_model == "bgr") or (src_model == "bgr" and dest_model == "rgb"):
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    else: 
      raise Exception('ERROR:', 'src_model or dest_model not implemented')

    return frame

### Загружаем в память датасеты для обучения и тестирования

In [ ]:
images_ann = open("./data/annotated_logos/zeros/classes.txt").read().split()

get_image_class = {}

for img_a in images_ann:
    image_name, cl = img_a.split(";")
    get_image_class[image_name] = int(cl)

In [ ]:
train_dataset_dir = "./data/annotated_logos/zeros/train/"
test_dataset_dir = "./data/annotated_logos/zeros/test/"

In [ ]:
# Обучающие данные
# Библиотеки для классификатора


# Обучающие данные
X = np.array([]) # Массив тестов — признаки
y = np.array([]) # Массив тестов — ответы
# Тестировочные данные
X_test = np.array([]) # Массив тестов — признаки
y_test = np.array([]) # Массив тестов — ответы

# Тренировочные данные
for num_file in range(len(os.listdir(train_dataset_dir))):
    # Путь к очередному файлу
    image_name = os.listdir(train_dataset_dir)[num_file]
    path = train_dataset_dir+image_name
    # Пропускаем итерацию, если файл - не картинка
    if(path[-3:]!='jpg'):
        continue
    # Откроем файл
    img = cv2.imread(path)
    img = convert(img, 'bgr', 'rgb')
    # Отмасштабируем его
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    
    # Возьмем HOG в качестве признака
    HOG, HOG_img = GetHOG(img.copy(), settings)
    # Поместим HOG во входные данные
    if X.size == 0: # Проверим существования данных в массиве для соблюдения его размерности
        X = np.array([HOG])
    else:
        X = np.append(X, [HOG], axis = 0)
    # Поместим ответ к картинке
    

    # label = int(image_name.split(".")[0].split("_")[1]) + 1
    label = get_image_class[image_name] + 1
    y = np.append(y, label)

In [ ]:
# Тестировочные данные
for num_file in range(len(os.listdir(test_dataset_dir))):
    # Путь к очередному файлу
    image_name = os.listdir(test_dataset_dir)[num_file]
    path = test_dataset_dir+str(os.listdir(test_dataset_dir)[num_file])
    # Пропускаем итерацию, если файл - не картинка
    if(path[-3:]!='jpg'):
        continue
    # Откроем файл
    img = cv2.imread(path)
    img = convert(img, 'bgr', 'rgb')
    # Отмасштабируем его
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    
    # Возьмем HOG в качестве признака
    HOG, HOG_img = GetHOG(img.copy(), settings)
    # Поместим HOG во входные данные
    if X_test.size == 0: # Проверим существования данных в массиве для соблюдения его размерности
        X_test = np.array([HOG])
    else:
        X_test = np.append(X_test, [HOG], axis = 0)
    # Поместим ответ к картинке
    label = get_image_class[image_name] + 1
    y_test = np.append(y_test, label)

In [ ]:
(y == 1).sum(), (y == 2).sum()

(238, 968)

### Обучаем модель

In [ ]:
# Обучим предсказывающую модель

clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) # Предсказывающая модель
clf.fit(X, y) # Обучим модель на данных X и y
dump(clf, 'classifier.joblib') 

['classifier.joblib']

In [ ]:
clf = load('./classifier.joblib') 

### Протестируем модель

In [ ]:
# Функция теста точности модели
def TestPrediction(model, questions, answers):
    modelAnswers = clf.predict(questions)
    countRight = 0
    countAnswers = len(modelAnswers)
    for i in range(len(modelAnswers)):
        if int(modelAnswers[i]) == answers[i]:
            countRight += 1
    # Количество правильных ответов ко всем
    return float(countRight)/countAnswers

In [ ]:
# Выведем точность модели
print(f'Точность составляет: {TestPrediction(clf, X_test, y_test)*100} %')

Точность составляет: 96.66666666666667 %


### Проверим работу на конкретных примерах

In [ ]:
# Функция предсказания с обработкой входных данных
def predict(model, img):
    # Отмасштабируем изображение
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    
    # Возьмем HOG в качестве признака
    HOG, HOG_img = GetHOG(img.copy(), settings)
    return model.predict([HOG])

In [ ]:
prefix = train_dataset_dir
for image_idx in range(len(os.listdir(prefix))):
    # Укажем путь к картинке
    path = prefix+str(os.listdir(prefix)[image_idx])
    if(path[-3:]!="jpg"):
        print("Неверный формат файла")
    else:
        img = cv2.imread(path)
        img = convert(img, 'bgr', 'rgb')
        true_ans = get_image_class[os.listdir(prefix)[image_idx]] + 1
        if true_ans == 1:
            print(f'На картинке есть лого: {int(predict(clf, img))}\nНа самом деле: {true_ans}')
        # fig, m_axs = plt.subplots(1,1, figsize=(4,4))
        # ax1 = m_axs
        
        # ax1.set_title('Изображение')
        # ax1.imshow(img)

На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом деле: 1
На картинке есть лого: 1
На самом 

KeyboardInterrupt: ignored